# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test` __Do NOT commit the pickle file__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
    
    df = df.sample(5000, random_state=100).reset_index(drop=True)

    
    y = df['sellingprice']
    X = df.drop('sellingprice', axis=1)
    
    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
   
    while(True):
        yield df.sample(100).reset_index(drop=True)
        
gen = partial_df()

In [3]:
X_test, y_test = test_df()

In [4]:
X_test.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,saledate
0,2008,Chevrolet,Tahoe,LS,SUV,automatic,1gnfc13c38j114097,tn,2,100335.0,blue,black,"consolidated asset recovery systems, inc.",13500,Tue Dec 23 2014 10:00:00 GMT-0800 (PST)
1,2011,Infiniti,G Sedan,G37x,G Sedan,automatic,jn1cv6ar3bm411941,tn,4.3,36724.0,black,—,nissan infiniti lt,17950,Wed Mar 04 2015 02:30:00 GMT-0800 (PST)
2,2012,Toyota,RAV4,Base,suv,automatic,2t3bf4dv0cw239620,oh,2.6,22142.0,blue,gray,toyota financial services,15450,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
3,2014,Dodge,Avenger,SE,Sedan,automatic,1c3cdzab1en138859,mn,4.5,11893.0,blue,black,capital one auto finance,12200,Wed May 27 2015 03:30:00 GMT-0700 (PDT)
4,2011,Ford,Focus,SEL,Sedan,automatic,1fahp3hn9bw194095,nc,4.7,72731.0,gray,gray,parkway ford lincoln,8025,Tue Mar 03 2015 01:30:00 GMT-0800 (PST)


In [5]:
# each time you call this you will get a new slice of the dataframe.
df1=next(gen)
df1

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2009,Nissan,Maxima,3.5 SV,Sedan,NaN,1n4aa51ex9c826025,pa,2,76484.0,silver,black,remarketing by ge/flagship credit acceptance,12200,10800,Fri Feb 13 2015 01:00:00 GMT-0800 (PST)
1,2015,Ford,Taurus,Limited,Sedan,automatic,1fahp2f80fg118658,sc,1,5463.0,silver,tan,enterprise vehicle exchange / tra / rental / t...,23100,18300,Thu Jan 22 2015 04:00:00 GMT-0800 (PST)
2,2013,Toyota,Camry,SE,Sedan,automatic,4t1bf1fk1du269463,in,3.6,50044.0,silver,black,the hertz corporation,13750,13900,Thu Feb 19 2015 02:10:00 GMT-0800 (PST)
3,2012,Nissan,Altima,2.5,Sedan,automatic,1n4al2ap5cn529248,fl,4.5,25085.0,gray,gray,nissan-infiniti lt,12950,12600,Wed Mar 04 2015 01:00:00 GMT-0800 (PST)
4,2008,Mercedes-Benz,C-Class,C300 Sport,sedan,automatic,wddgf54x38f017553,ca,2.8,121656.0,gray,black,apexcars net,10950,10600,Wed Jun 03 2015 05:30:00 GMT-0700 (PDT)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2008,Chevrolet,Cobalt,Sport,Sedan,automatic,1g1am58b387177777,oh,4,100033.0,white,gray,dt credit corporation,4225,5500,Thu Feb 12 2015 01:00:00 GMT-0800 (PST)
96,2006,Chrysler,Town and Country,Limited,minivan,automatic,2a8gp64l56r915435,pa,2.7,114734.0,white,gray,champion mazda,3900,4700,Fri Jun 05 2015 02:00:00 GMT-0700 (PDT)
97,2006,Ford,F-150,XL,Regular Cab,automatic,1ftrf12w26na96724,md,3.5,160405.0,white,gray,darcars ford,2875,4200,Tue Feb 10 2015 01:30:00 GMT-0800 (PST)
98,2012,Toyota,Corolla,S,Sedan,automatic,5yfbu4ee8cp005062,tx,3.9,70708.0,red,—,chrysler capital,10150,10900,Wed Feb 18 2015 02:00:00 GMT-0800 (PST)


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

# load data into a pandas dataframe
#df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

df1.dropna(inplace=True)

for column in df1.columns:
        if df1[column].dtype == 'object':
            encoder = LabelEncoder()
            df1[column] = encoder.fit_transform(df1[column])
            
# define input features and target variable
X = df1.drop('sellingprice', axis=1)
y = df1['sellingprice']


# split into training and testing sets
X_train, X_testing, y_train, y_testing = train_test_split(X, y, test_size=0.2)

# define the preprocessing pipeline
preprocessor = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# fit the preprocessing pipeline on the training data and transform both training and testing data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_testing)

# create SGDRegressor object
regressor = SGDRegressor()

# set hyperparameters
batch_size = 32
num_epochs = 10

# loop over the dataset for the specified number of epochs
for epoch in range(num_epochs):
    # loop over the data in mini-batches
    for i in range(0, len(X_train_preprocessed), batch_size):
        X_batch = X_train_preprocessed[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        # train the model on the mini-batch
        regressor.partial_fit(X_batch, y_batch)

# evaluate the model on test data
score = regressor.score(X_test_preprocessed, y_testing)
print("R^2 score:", score)

R^2 score: 0.6709149336692752


In [7]:
import pickle

# train your model
regressor.fit(X_train, y_train)

# save the trained model to a file
with open('my_model.pkl', 'wb') as f:
    pickle.dump(regressor, f)


In [8]:
# load the saved model from a file
with open('my_model.pkl', 'rb') as f:
    load_model = pickle.load(f)

In [10]:
# preprocess the test data using the same steps as the training data
X_test_processed = preprocessor.transform(X_test)

In [11]:
# select only numeric columns from the preprocessed test data
#X_test_numeric = X_test.select_dtypes(include=['float64', 'int64'])
#X_train_numeric = X_train.select_dtypes(include=['float64', 'int64'])

# get the feature names used during training
#feature_names = list(X_train_numeric.columns)

#X_test_numeric.columns = feature_names

import warnings
warnings.filterwarnings('ignore')

# make predictions on the preprocessed test data
y_pred = load_model.predict(X_test_processed)